In [1]:
%pip install trl transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, Dataset
import random

# Load the TACO dataset
raw_dataset = load_dataset("BAAI/TACO", split="train")

# Build a pseudo reward dataset by pairing responses
reward_data = []

for i in range(0, len(raw_dataset) - 1, 2):
    ex1 = raw_dataset[i]
    ex2 = raw_dataset[i+1]

    prompt1 = f"{ex1['instruction']}\n{ex1['input']}" if ex1["input"] else ex1["instruction"]
    prompt2 = f"{ex2['instruction']}\n{ex2['input']}" if ex2["input"] else ex2["instruction"]

    # Ensure same instruction to simulate preference
    if prompt1.strip() != prompt2.strip():
        continue

    # Randomly assign which output is preferred
    if random.random() < 0.5:
        chosen, rejected = ex1["output"], ex2["output"]
    else:
        chosen, rejected = ex2["output"], ex1["output"]

    reward_data.append({
        "prompt": prompt1.strip(),
        "chosen": chosen.strip(),
        "rejected": rejected.strip()
    })

# Convert to Hugging Face Dataset
reward_dataset = Dataset.from_list(reward_data)


/opt/homebrew/anaconda3/envs/competitiveLLM/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyError: 'input'